In [1]:
from comet_ml import Experiment
import torch
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import tqdm
import numpy as np
from loguru import logger
import json
from livelossplot import PlotLosses

In [2]:
from torch import nn
import torch.nn.functional as F

In [3]:
logger.remove()
logger.add("log.txt")

1

In [4]:
DEVICE = "cuda:1"

## Load models for vector generation

## Load data

### Read vectors

In [5]:
embeddings = np.load("data/vectors.npy")

In [6]:
indices = np.arange(embeddings.shape[0]//3)

train_idx, test_idx = train_test_split(indices, test_size=0.9, random_state=42)

train_data = [embeddings[train_idx*3], embeddings[train_idx*3+1], embeddings[train_idx*3+2]]
test_data = [embeddings[test_idx*3], embeddings[test_idx*3+1], embeddings[test_idx*3+2]]

print(train_data[0].shape)
print(test_data[0].shape)

(9997, 768)
(89978, 768)


## Measure cosine similarity differences between pos and neg

In [50]:
def calculate_metrics(method, data, name):
    orig_shape = data[0].shape[1]
    pos_similarity = []
    neg_similarity = []

    queue = []
    anchors = method(data[0])
    positives = method(data[1])
    negatives = method(data[2])
   
    for i in tqdm.trange(len(anchors)):
        query, pos, neg = anchors[i], positives[i], negatives[i]
        pos_similarity.append(cosine_similarity([query], [pos])[0][0])
        neg_similarity.append(cosine_similarity([query], [neg])[0][0])

    pos_similarity = np.array(pos_similarity)
    neg_similarity = np.array(neg_similarity)
    
    ratio = np.sum(pos_similarity>neg_similarity)/len(pos_similarity)
    mean_diff = pos_similarity.mean()-neg_similarity.mean()
    mean_pos_sim = pos_similarity.mean()
    
    reduction_rate = orig_shape/len(anchors[0])
    return pd.DataFrame.from_dict([{
                        "ratio": ratio,
                        "mean_pos_sim": mean_pos_sim,
                        "mean_diff": mean_diff,
                        "method": name,
                        "reduction_rate": reduction_rate
                    }])

## Benchmarks

In [8]:
metrics = []

### Baseline

In [120]:
metrics.append(calculate_metrics(lambda x: x, test_data, "CLS vector"))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29993/29993 [00:11<00:00, 2722.30it/s]


In [ ]:
# lda_metrics = []
# embeddings = []

# train_data = data.sample(1000, random_state=42)

# for i in tqdm.trange(0, train_data.shape[0], 32):
#     embeddings.extend(embed(

In [40]:
dim = 64

# for idx in range(0, train.shape

In [42]:
for

,query,pos,neg
15773,what does dsm stand for,"In psychology and psychiatry, DSM stands for t...","military mk, what does mark stand for in milit..."
35290,what is social competence in early childhood,Social competence in early childhood is as a s...,The Early Years Learning Framework (EYLF) is d...
358404,stonefield construction phone number,A: The phone number for Stonefield Constructio...,The Building Official reviews applications for...
381806,what book did adam smith write,"Adam Smith FRSA was a Scottish economist, phil...",Was Adam Smith laissez-faire? What did it mean...
244560,how long can you take naltrexone for,You and your doctor have to decide this. Most ...,"Naloxone, also known as Narcan among other nam..."
...,...,...,...
178265,what happens in the first stage of seed germin...,Stage One - Hydration and Metabolism. In the f...,Germination is the process in which the seeds ...
168368,aberdeen airport code,Aberdeen International Airport (Scottish Gaeli...,Welcome to USZip.com. USZip.com USZip.com is a...
314866,how old till baby can sleep on stomach,You may find yourself wondering if your baby c...,Third Month Baby Milestones: Sleep. Your 3-mon...
472372,what are the sisters last names in in the time...,In the Time of the Butterflies Characters. BÃ©...,The nine brightest stars of the Pleiades are n...


### Averaging

In [115]:
def average_vec(dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, dim)
        x = x.mean(axis=1).reshape(-1, 1)
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

In [118]:
for rate in [4, 8, 16, 32, 64, 128]:
    method = average_vec(rate)
    metrics.append(calculate_metrics(method, test_data, f"Average parts ({rate})"))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29993/29993 [00:10<00:00, 2991.96it/s]


In [122]:
pd.concat(metrics).sort_values("ratio").to_csv("results/benchmarks.csv")

### Generic autoencoder

In [102]:
def get_model(input_dim, output_dim):
    encoder = nn.Sequential(
                    nn.Linear(input_dim, 16),
                    nn.ReLU(),
                    nn.Linear(16, output_dim),
            )
    decoder = nn.Sequential(
                    nn.Linear(output_dim, 16),
                    nn.ReLU(),
                    nn.Linear(16, input_dim)
            )
    autoenc = nn.Sequential(
                    encoder,
                    decoder
                )
    return autoenc

def test_autoenc(model, dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, dim)
        x = torch.from_numpy(x).to(torch.float32)
        x = model[0](x).cpu().detach().numpy()
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

In [103]:
data = torch.tensor(train_data).reshape(-1, 768)
train_loader = torch.utils.data.DataLoader(data.to(torch.float32), batch_size=32, shuffle=True)

batch_size = 32

def train_autoenc(model, dim, criterion, epochs=1, params={}):
    optim = torch.optim.Adam(model.parameters(), 3e-4)
    
    experiment = Experiment(
    api_key="sQnInDpH3eOHuFvstNS3jG5jb",
    project_name="vector-compression",
    workspace="wwydmanski",
    display_summary_level=0,
    auto_metric_logging=False
    )
    experiment.add_tag("autoencoder")
    experiment.log_parameters(params)
    
    for epoch in range(1, epochs+1):
        with tqdm.tqdm(train_loader) as t:
            total_loss = []

            for step, batch in enumerate(t):
                x = batch.reshape(-1, dim)
                batch_loss = []
                for idx in range(0, len(x), batch_size):
                    minibatch = x[idx:idx+batch_size]

                    optim.zero_grad()

                    y_hat = model(minibatch)
                    loss = criterion(minibatch, y_hat)
                    loss.backward()
                    optim.step()

                    batch_loss.append(loss.item())

                experiment.log_metric("batch_loss_", np.mean(batch_loss), step=step, epoch=epoch)

            experiment.log_metric("loss_", np.mean(total_loss), step=step, epoch=epoch)

        res = calculate_metrics(test_autoenc(model, dim), train_data, f"Autoencoder")
        print(res)
        experiment.log_metric("Train accuracy", res["ratio"].values[0], step=epoch*step)
        experiment.log_metric("Train mean positive similarity", res["mean_pos_sim"].values[0], step=epoch*step)
        experiment.log_metric("Train mean difference", res["mean_diff"].values[0], step=epoch*step)
        
    res = calculate_metrics(test_autoenc(model, dim), test_data, f"Autoencoder")
    print(res)
    experiment.log_metric("Test accuracy", res["ratio"].values[0], step=epoch*step)
    experiment.log_metric("Test mean positive similarity", res["mean_pos_sim"].values[0], step=epoch*step)
    experiment.log_metric("Test mean difference", res["mean_diff"].values[0], step=epoch*step)
        
    experiment.log_parameter("Reduction rate", res["reduction_rate"].values[0])
    experiment.end()

In [108]:
metrics = []

In [113]:
hinge_margin = torch.tensor(0)
gamma = 1
def var_loss(x):
    var = torch.sqrt(x.var(0)+1e-6)
    loss = torch.maximum(hinge_margin, gamma-var).mean()
    return loss


dim=32
output_dim = 8
lr = 3e-4

for shape in [(128, 8), (256, 8), (256, 4), (128, 4)][::-1]:
    params = {"Input dim": shape[0], "Output dim": shape[1], "LR": lr}
    print(params)
    model = get_model(shape[0], shape[1])

    optim = torch.optim.Adam(model.parameters(), lr)
    criterion_ = nn.CosineSimilarity()
    criterion = lambda x, y: -criterion_(x, y).mean() - 0.1*var_loss(y)

    train_autoenc(model, shape[0], criterion, epochs=4, params=params)
    metrics.append(calculate_metrics(test_autoenc(model, shape[0]), test_data, f"Autoencoder"))

{'Input dim': 128, 'Output dim': 4, 'LR': 0.0003}


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/vector-compression/0a17a85eb2344674af0605d36d02285d

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [00:07<00:00, 121.08it/s]
/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████

      ratio  mean_pos_sim  mean_diff       method  reduction_rate
0  0.872762      0.861803   0.113987  Autoencoder            32.0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [00:07<00:00, 129.90it/s]
/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9997/9997 [00:03<00:00, 2571.72it/s]


      ratio  mean_pos_sim  mean_diff       method  reduction_rate
0  0.854356      0.843139    0.12715  Autoencoder            32.0


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [00:07<00:00, 129.37it/s]
/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9997/9997 [00:04<00:00, 2484.27it/s]


      ratio  mean_pos_sim  mean_diff       method  reduction_rate
0  0.913074      0.955115    0.03862  Autoencoder            16.0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89978/89978 [00:36<00:00, 2496.04it/s]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications


      ratio  mean_pos_sim  mean_diff       method  reduction_rate
0  0.908511      0.954903    0.03825  Autoencoder            16.0


COMET INFO: Uploading 1 metrics, params and output messages
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89978/89978 [00:35<00:00, 2558.71it/s]


In [124]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/autoencoder.csv")

cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.751017,0.961556,0.027461,Autoencoder,64.0
0,0.837927,0.960247,0.027499,Autoencoder,32.0
0,0.909922,0.970591,0.023799,Autoencoder,16.0
0,0.935818,0.962036,0.031864,Autoencoder,8.0
0,0.958501,0.987163,0.010953,Autoencoder,2.0


### PCA

In [ ]:
metrics = []

In [126]:
from sklearn.decomposition import PCA

In [19]:
def test_pca(model, input_dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, input_dim)
        x = model.transform(x)
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

def get_pca(input_dim, output_dim):
    model = PCA(output_dim)
    data = train_data[0].reshape(-1, input_dim)
    model.fit(data)
    return model

In [139]:
model = get_pca(64, 8)

In [142]:
for input_dim in [8, 16, 32, 64, 128, 768]:
    for output_dim in [2, 4, 8, 16, 32, 64, 128]:
        if input_dim <= output_dim:
            continue
            
        model = get_pca(input_dim, output_dim)
        metrics.append(calculate_metrics(test_pca(model, input_dim), test_data, f"PCA ({input_dim}x{output_dim})"))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89978/89978 [00:32<00:00, 2789.97it/s]


In [144]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/PCA.csv")

cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.610516,0.811125,0.125235,PCA (768x2),384.0
0,0.683045,0.841940,0.127949,PCA (768x4),192.0
0,0.749561,0.841689,0.138854,PCA (768x8),96.0
0,0.810209,0.735790,0.207031,PCA (128x2),64.0
0,0.829080,0.832002,0.157467,PCA (768x16),48.0
0,0.879971,0.728551,0.222813,PCA (64x2),32.0
0,0.895930,0.815812,0.179110,PCA (768x32),24.0
0,0.920536,0.734346,0.224905,PCA (32x2),16.0
0,0.935962,0.802815,0.198501,PCA (768x64),12.0
0,0.944464,0.738165,0.221835,PCA (64x8),8.0


### UMAP

In [9]:
metrics = []

In [23]:
from cuml.manifold import UMAP

In [37]:
def get_umap(input_dim, output_dim):
    model = UMAP(n_components=output_dim)
    data = train_data[0][:10000].reshape(-1, input_dim)
    model.fit(data)
    return model

In [38]:
model = get_umap(128, 16)

In [ ]:
for input_dim in [16]:
    for output_dim in [2, 4, 8]:
        if input_dim <= output_dim:
            continue
            
        model = get_umap(input_dim, output_dim)
        metrics.append(calculate_metrics(test_pca(model, input_dim), test_data, f"UMAP ({input_dim}x{output_dim})"))

 60%|█████████████████████████████████████████████████████████████████████████████████████████████████                                                                | 54275/89978 [00:20<00:13, 2678.39it/s]

In [35]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/UMAP.csv")

cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.550879,0.668035,0.040766,UMAP (768x2),384.0
0,0.632088,0.911761,0.057221,UMAP (768x4),192.0
0,0.651104,0.957159,0.033607,UMAP (768x8),96.0
0,0.544500,0.846501,0.028204,UMAP (128x2),64.0
0,0.653393,0.968231,0.024588,UMAP (768x16),48.0
0,0.644124,0.954982,0.030882,UMAP (128x4),32.0
0,0.650470,0.975806,0.017704,UMAP (768x32),24.0
0,0.673565,0.965916,0.033695,UMAP (128x8),16.0
0,0.614295,0.824949,0.034135,UMAP (768x64),12.0
0,0.885116,0.629663,0.168973,UMAP (32x4),8.0


### LocallyLinearEmbedding

In [8]:
from sklearn.manifold import LocallyLinearEmbedding

In [16]:
batch_size = 512
def test_lle(model, input_dim):
    def _inner(embeddings):
        res = []
        x = embeddings.reshape(-1, input_dim)
        
        for i in tqdm.trange(0, len(x), batch_size):
            res.append(model.transform(x[i:i+batch_size]))

        x = np.concatenate(res)
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

def get_lle(input_dim, output_dim):
    model = LocallyLinearEmbedding(n_components=output_dim, n_jobs=20)
    data = train_data[0].reshape(-1, input_dim)[:5000]
    model.fit(data)
    return model

In [17]:
metrics = []

In [ ]:
for input_dim in [8, 16, 32, 64, 128, 768][::-1]:
    for output_dim in [2, 4, 8, 16, 32, 64, 128]:
        if input_dim <= output_dim:
            continue
            
        model = get_lle(input_dim, output_dim)
        metrics.append(calculate_metrics(test_lle(model, input_dim), test_data, f"LocallyLinearEmbedding ({input_dim}x{output_dim})"))

 43%|███████████████████████████████████████████████████████████████████████▋                                                                                              | 910/2109 [01:08<01:31, 13.16it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 40%|██████████████████████████████████████████████████████████████████▎                                                                                                  | 1696/4218 [02:10<03:13, 13.02it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (ms

In [21]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/LocallyLinear.csv")

cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.625553,0.839529,0.093303,LocallyLinearEmbedding (768x2),384.0
0,0.660684,0.851210,0.100017,LocallyLinearEmbedding (768x4),192.0
0,0.675387,0.831586,0.115023,LocallyLinearEmbedding (768x8),96.0
0,0.637178,0.698145,0.114979,LocallyLinearEmbedding (128x2),64.0
0,0.696837,0.817764,0.128250,LocallyLinearEmbedding (768x16),48.0
0,0.754229,0.446227,0.186492,LocallyLinearEmbedding (64x2),32.0
0,0.709951,0.781216,0.152467,LocallyLinearEmbedding (768x32),24.0
0,0.799118,0.435487,0.185842,LocallyLinearEmbedding (64x4),16.0
0,0.723888,0.748783,0.173493,LocallyLinearEmbedding (768x64),12.0
0,0.856932,0.447548,0.177353,LocallyLinearEmbedding (16x2),8.0



## Whitening Sentence Representations 

In [45]:
def compute_kernel_bias(input_dim):
    """
    y = (x + bias).dot(kernel)
    """
    vecs = train_data[0].reshape(-1, input_dim)
    # vecs = np.concatenate(vecs, axis=0)
    mu = vecs.mean(axis=0, keepdims=True)
    cov = np.cov(vecs.T)
    u, s, vh = np.linalg.svd(cov)
    W = np.dot(u, np.diag(1/np.sqrt(s)))
    return W, -mu

def transform_and_normalize(vecs, kernel, bias):
    if not (kernel is None or bias is None):
        vecs = (vecs + bias).dot(kernel)
    normalized=vecs / (vecs**2).sum(axis=1, keepdims=True)**0.5
    return normalized

def test_whitening(kernel, bias,input_dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, input_dim)
        x = transform_and_normalize(x,kernel=kernel,bias=bias)
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

In [48]:
metrics = []

In [51]:
for input_dim in [32, 64, 128, 768]:
    for output_dim in [2, 4, 8, 16, 32, 64, 128]:
        if input_dim <= output_dim:
            continue
        kernel, bias = compute_kernel_bias(input_dim)
        kernel = kernel[:, :output_dim]
        metrics.append(calculate_metrics(test_whitening(kernel,bias,input_dim), test_data, f"Whitening ({input_dim}x{output_dim})"))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 89978/89978 [00:26<00:00, 3386.55it/s]


In [55]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/Whitening.csv")
cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.610883,0.810334,0.124641,Whitening (768x2),384.0
0,0.683523,0.840056,0.128928,Whitening (768x4),192.0
0,0.751950,0.838032,0.142059,Whitening (768x8),96.0
0,0.748850,0.652865,0.199488,Whitening (128x2),64.0
0,0.833070,0.826492,0.163008,Whitening (768x16),48.0
0,0.839594,0.703681,0.209091,Whitening (128x4),32.0
0,0.902454,0.806293,0.190766,Whitening (768x32),24.0
0,0.902409,0.724704,0.213697,Whitening (128x8),16.0
0,0.937863,0.793159,0.210244,Whitening (768x64),12.0
0,0.938040,0.735265,0.217495,Whitening (128x16),8.0
